# 1. Import Packages

In [1]:
import sys, os
import pandas as pd
import numpy as np
from datetime import timedelta
import datetime
from timer import date_to_date, time_to_delta

# 2. Load Files
데이터셋이 저장되어 있는 nielsen_data 폴더를 path 변수로 지정, 해당경로 밑에 있는 파일명을 불러와 files 변수에 list 형태로 저장

In [ ]:
path = 'nielsen_data_new_20170328/T1'
files = os.listdir(path)

각각의 텍스트 파일을 읽어와 text에 line-by-line으로 저장

In [27]:
text = []
for file in files:
    print('Reading %s' % file)
    with open(os.path.join(path, file), 'rb') as f:
        text += f.readlines()
print("\n========%s logs are loaded========\n" % str(len(text)))

Reading 20150817.txt
Reading 20150818.txt
Reading 20150819.txt
Reading 20150820.txt
Reading 20150821.txt
Reading 20150822.txt
Reading 20150823.txt
Reading 20150824.txt
Reading 20150825.txt
Reading 20150826.txt
Reading 20150827.txt
Reading 20150828.txt
Reading 20150829.txt
Reading 20150830.txt

========908408 logs are loaded========



# 3. Data Preprocessing
각 line(log)의 encoding을 변환('euc-kr' -> 'utf-8'), 각 line 끝에 new line을 strip 함수로 제거하고 구분자인 '^'로 split하여 parsed에 저장

In [28]:
parsed = list(map(lambda line: line.decode('euc-kr').strip('\r\n').split('^'), text))
print('Every line is successfully parsed.')

Every line is successfully parsed.


In [6]:
pd.to_datetime(parsed[0][0])
# list(map(lambda x: datetime.time(int(x[0])) + datetime.timedelta(int(x[10]), '%H%M%S'), sample))
print(datetime.datetime(2016, 8,23)+datetime.timedelta(hours = 26, minutes = 43, seconds = 11))

2016-08-24 02:43:11


data frame에서 사용할 column 이름(제공받은 엑셀 파일에서 복사)을 split 하여 list 형태로 cols에 저장

In [29]:
cols = "일자	패널가구ID	개인ID	가중치	성별	연령	직업	학력	소득	채널	시청시작시간	시청종료시간	프로그램시청시간	프로그램명	프로그램편성시작시간	프로그램편성종료시간	프로그램장르"
cols = cols.split()

parsed에 저장된 로그들을 data로, cols에 저장된 이름을 columns으로 하는 pandas data frame 생성 : raw_df

In [30]:
raw_df = pd.DataFrame(data=parsed, columns=cols)

## 3.1. Convert string to datetime
raw_df에 string으로 표현되어 있는 각종 시간에 관련된 column을 시간 연산이 가능하도록 type 변경 함수 작성

date_to_date: '20160823'의 형태로 되어 있는 데이터를 2016년 8월 23일로 변경

time_to_delta: 24시 이후는 다음날로 처리하기 위하여 timedelta의 형태('265745'의 형태로 되어 있는 데이터를 26시간 57분 45초)로 변경

In [9]:
def date_to_date(date):
    year, month, day = map(int, [date[0:4], date[4:6], date[6:8]])
    return datetime.datetime(year, month, day)

def time_to_delta(time):
    hours, minutes, seconds = map(int, [time[0:2], time[2:4], time[4:6]])
    return datetime.timedelta(hours=hours, minutes=minutes, seconds=seconds)

# print('===== Test Code =====')
# d = input('Type original date in string(YYYYMMDD) : ')
# t = input('Type original time in string(HHMMSS) : ')
# print('Typed string \'%s\' and \'%s\' is converted to %s' % (d, t, date_to_date(d) + time_to_delta(t)))
# print('===== Test End =====')

작성한 함수를 활용하여 시청시작/종료시간 및 프로그램시작/종료시간을 일자와 합산하여 별도의 column 생성

In [31]:
raw_df['view_start'] = raw_df.일자.apply(date_to_date) + raw_df.시청시작시간.apply(time_to_delta)
raw_df['view_end'] = raw_df.일자.apply(date_to_date) + raw_df.시청종료시간.apply(time_to_delta)
# raw_df['program_start'] = raw_df.일자.apply(date_to_date) + raw_df.프로그램편성시작시간.apply(time_to_delta)
# raw_df['program_end'] = raw_df.일자.apply(date_to_date) + raw_df.프로그램편성종료시간.apply(time_to_delta)

프로그램시청시간(duration) = 시청종료시간(view_end) - 시청시작시간(view_start)

In [11]:
raw_df['duration'] = raw_df.view_end - raw_df.view_start + timedelta(seconds = 1)
# raw_df['duration_sec'] = raw_df.duration.apply(timedelta.total_seconds)

## 3.2. Deal with NaN
데이터가 누락된 빈 문자열('')을 numpy의 nan으로 설정하여 nan이 포함된 행(axis = 0)을 drop

In [12]:
tidy_df = raw_df.replace('', np.nan, regex=True).dropna(axis = 0)
# tidy_df.프로그램시청시간 = tidy_df.프로그램시청시간.astype(int)

전처리 과정이 완료된 데이터 상위 10행 예시

In [73]:
# tidy_df.head(10)
tidy_df.ix[(tidy_df.일자 == '20150817') & (tidy_df.패널가구ID == '1401639') & (tidy_df.개인ID == 'aa'), 9:]

,채널,시청시작시간,시청종료시간,프로그램시청시간,프로그램명,프로그램편성시작시간,프로그램편성종료시간,프로그램장르,view_start,view_end,duration
0,2,080900,082359,883,인간극장,075006,082342,010005005,2015-08-17 08:09:00,2015-08-17 08:23:59,00:15:00
2,2,120200,120259,60,KBS뉴스12,115948,125917,005005088,2015-08-17 12:02:00,2015-08-17 12:02:59,00:01:00
3,2,120800,120959,120,KBS뉴스12,115948,125917,005005088,2015-08-17 12:08:00,2015-08-17 12:09:59,00:02:00
4,2,124200,124359,120,KBS뉴스12,115948,125917,005005088,2015-08-17 12:42:00,2015-08-17 12:43:59,00:02:00
5,2,132900,133059,120,역사저널그날<재>,125918,134725,005010010,2015-08-17 13:29:00,2015-08-17 13:30:59,00:02:00
6,2,135000,135259,178,별별가족<재>,134846,135257,010005005,2015-08-17 13:50:00,2015-08-17 13:52:59,00:03:00
8,2,143400,143559,120,한국인의밥상<재>,140030,145339,010005010,2015-08-17 14:34:00,2015-08-17 14:35:59,00:02:00
10,2,160500,160759,180,KBS오늘의경제,155901,160809,010010020,2015-08-17 16:05:00,2015-08-17 16:07:59,00:03:00
12,2,163800,163959,120,걸어서세계속으로<재>,160927,165752,010005010,2015-08-17 16:38:00,2015-08-17 16:39:59,00:02:00
14,3,082400,083459,660,KBS8아침뉴스타임,075738,085504,005005088,2015-08-17 08:24:00,2015-08-17 08:34:59,00:11:00


In [36]:
# t = tidy_df.groupby(['일자', '패널가구ID', '개인ID']).시청시작시간.apply(list).reset_index(name = 't')

In [70]:
# cont = lambda row: row.shift(1) == row

In [79]:
# tidy_df.groupby(['일자', '패널가구ID', '개인ID']).시청시작시간 == 
tidy_df.groupby(['일자', '패널가구ID', '개인ID']).get_group['20160817']

TypeError: 'method' object is not subscriptable

In [114]:
# for key in t.groups.keys():
#     print(t.groups[key])
timedelta(23, 17, 59) - timedelta(21, 54, 0)

datetime.timedelta(1, 86363, 59)

# 4. Analysis

## 4.1. Daily program repertoire of each person 

tidy_df 데이터 프레임을 일자별, 개인별로 각 프로그램 시청시간을 합산하여 새로운 column(duration_sum)으로 지정하여 program_view_sum에 저장

In [13]:
program_view_sum = tidy_df.groupby(['일자', '패널가구ID', '개인ID', '프로그램명'])['duration'].sum().reset_index(name = 'duration_sum')

program_view_sum에서 일자별, 개인별로 시청한 프로그램 중 시청시간 threshold 이상인 것만 subset으로 filtered에 저장

In [14]:
filtered = program_view_sum[program_view_sum.duration_sum > datetime.timedelta(minutes = 15)]

In [15]:
type(program_view_sum)

pandas.core.frame.DataFrame

threshold 이상 시청한 프로그램을 대상으로 일자별, 개인별로 시청한 프로그램을 list 형태로 저장하고 daily_program_repertoire에 저장

In [16]:
daily_program_repertoire = filtered.groupby(['일자', '패널가구ID', '개인ID'])['프로그램명'].apply(list)
daily_program_repertoire.head(20)

일자        패널가구ID   개인ID
20150817  1401639  aa      [가족을지켜라, 그래도푸르른날에, 나혼자산다, 베를린-2부, 벤자민버튼의시간은거꾸로...
                   ab      [MBC뉴스데스크, 가족을지켜라, 그래도푸르른날에, 나혼자산다, 벤자민버튼의시간은거...
          1401643  aa      [SBS월화드라마(미세스캅), 광복특별기획학교교육백년사, 달라졌어요, 생활의달인, ...
          1401646  aa      [2TV저녁, 6시내고향, MBC뉴스데스크, SBS월화드라마(미세스캅), 우리말겨루...
                   ab      [2TV저녁, 6시내고향, MBC뉴스데스크, SBS월화드라마(미세스캅), 우리말겨루...
          1401694  aa                 [돌아온황금복, 스콜피온, 일일드라마(울지않는새<본>), 주말엔영화]
                   ab      [2015메이저리그, 2015메이저리그<생>, 마지막황제, 미스터리음악쇼복면가왕, ...
          1401699  aa      [JTBC뉴스룸, TV조선뉴스9, TV주치의닥터지바고<재>, 나는몸신이다<재>, 낭...
          1401700  aa                             [생활의달인, 안녕하세요, 창사특별기획(화정)]
                   ab                                   [KBS뉴스광장1부, 모닝와이드2부]
                   ac      [JTBC뉴스현장, 금토드라마(오나의귀신님), 금토미니시리즈(라스트<재>), 도둑들...
          1401710  aa      [곰디와친구들, 놀이터구조대뽀잉<재>, 두다다쿵<재>, 모닝와이드3부, 바오밥섬의파...
          1401714  aa                               

In [17]:
lambda x: x[2], daily_program_repertoire

(<function __main__.<lambda>>, 일자        패널가구ID   개인ID
 20150817  1401639  aa      [가족을지켜라, 그래도푸르른날에, 나혼자산다, 베를린-2부, 벤자민버튼의시간은거꾸로...
                    ab      [MBC뉴스데스크, 가족을지켜라, 그래도푸르른날에, 나혼자산다, 벤자민버튼의시간은거...
           1401643  aa      [SBS월화드라마(미세스캅), 광복특별기획학교교육백년사, 달라졌어요, 생활의달인, ...
           1401646  aa      [2TV저녁, 6시내고향, MBC뉴스데스크, SBS월화드라마(미세스캅), 우리말겨루...
                    ab      [2TV저녁, 6시내고향, MBC뉴스데스크, SBS월화드라마(미세스캅), 우리말겨루...
           1401694  aa                 [돌아온황금복, 스콜피온, 일일드라마(울지않는새<본>), 주말엔영화]
                    ab      [2015메이저리그, 2015메이저리그<생>, 마지막황제, 미스터리음악쇼복면가왕, ...
           1401699  aa      [JTBC뉴스룸, TV조선뉴스9, TV주치의닥터지바고<재>, 나는몸신이다<재>, 낭...
           1401700  aa                             [생활의달인, 안녕하세요, 창사특별기획(화정)]
                    ab                                   [KBS뉴스광장1부, 모닝와이드2부]
                    ac      [JTBC뉴스현장, 금토드라마(오나의귀신님), 금토미니시리즈(라스트<재>), 도둑들...
           1401710  aa      [곰디와친구들, 놀이터구조대뽀잉<재>, 두다다쿵<재>, 모닝와이드3부, 바오밥섬의파...
         

In [18]:
# sample_repertoire = list(pd.DataFrame(daily_program_repertoire).ix['20150827', '1401639', 'aa'])

In [19]:
daily_program_repertoire.to_csv('output/output.csv')